# Data Filtering and Preprocessing

This file takes the raw data for DASS and adds 5 new columns categorizing the data. Then, the data is filtered by certain conditions, and certain regions and unnecessary columns are dropped. 

# 1. Data Filtering

Import the necessary libraries and define helper functions to categorize country, age, and DASS-42 anxiety score into either string or numeric values. 

In [4]:
import os
import json
import pandas as pd
import pycountry_convert as pc

data_folder = "./data"

In [5]:
def encode_country(row):
    # Encode country into three major regions (east, west, other)
    country_code = row["country"]
    try:
        if country_code and country_code != "NONE":
            continent_name = pc.country_alpha2_to_continent_code(country_code)
            if continent_name == "AS":
                region_name = "east"
            elif continent_name in ["NA", "EU", "OC"]:
                region_name = "west"
            else:
                region_name = "other"
        else:
            region_name = ""
    except:
        region_name = ""
    return region_name


def encode_continent(row):
    # Encode country into three major regions (east, west, other)
    country_code = row["country"]
    try:
        if country_code and country_code != "NONE":
            continent_name = pc.country_alpha2_to_continent_code(country_code)
        else:
            continent_name = ""
    except:
        continent_name = ""
    return continent_name


def encode_age(row):
    # Encode age into groups
    age = int(row["age"])
    if age < 18:
        agegroup = 0
    elif age < 28:
        agegroup = 1
    elif age < 38:
        agegroup = 2
    elif age < 48:
        agegroup = 3
    elif age < 58:
        agegroup = 4
    elif age < 68:
        agegroup = 5
    else:
        agegroup = 6
    return agegroup


def calc_anx(row):
    # Calculate DASS-42 anxiety score
    with open(os.path.join(data_folder, "dass42_qcategories.json"), "r") as f:
        categories = json.load(f)
    anxiety_questions = [key for key in categories if categories[key] == "anxiety"]

    score = 0
    for qnum in anxiety_questions:
        score += int(row["Q{}A".format(qnum)])
    return score - len(anxiety_questions)


def categorize(row):
    # Classify as positive or negative (high or low) status based on threshold
    with open(os.path.join(data_folder, "dass42_scoring.json"), "r") as f:
        scoring = json.load(f)
    threshold = scoring["anxiety_score"]["severe"]["min"]  # moderate
    return (1 if row["anxiety_score"] >= threshold else 0)

Load the data and the helper functions will be called on the data to categorise the data into new columns.

In [ ]:
dataset = pd.read_csv("./data/data.csv", sep='\t')
dataset["agegroup"] = dataset.apply(lambda row: encode_age(row), axis=1)
dataset["continent"] = dataset.apply(lambda row: encode_continent(row), axis=1)
dataset["region"] = dataset.apply(lambda row: encode_country(row), axis=1)
dataset["anxiety_score"] = dataset.apply(lambda row: calc_anx(row), axis=1)
dataset["anxiety_status"] = dataset.apply(lambda row: categorize(row), axis=1)

Run the cells below to see an overview of the data before filtering. 

In [ ]:
print("Before filtering:")
print(dataset['gender'].value_counts())
print(dataset['age'].value_counts())
print(dataset['age'].mean(), dataset['age'].std())
print(dataset['continent'].value_counts())
print(dataset['region'].value_counts())
print(dataset['agegroup'].value_counts())
print(dataset['anxiety_status'].value_counts())

In [ ]:
print("\nBreakdown by continent:")
df1 = dataset[dataset['continent'] == 'AS']
print(df1['age'].mean(), df1['age'].std())
print(df1['gender'].value_counts())
print(df1['anxiety_status'].value_counts())

df2 = dataset[dataset['continent'] == 'NA']
print(df2['age'].mean(), df2['age'].std())
print(df2['gender'].value_counts())
print(df2['anxiety_status'].value_counts())

df3 = dataset[dataset['continent'] == 'EU']
print(df3['age'].mean(), df3['age'].std())
print(df3['gender'].value_counts())
print(df3['anxiety_status'].value_counts())

df4 = dataset[dataset['continent'] == 'SA']
print(df4['age'].mean(), df4['age'].std())
print(df4['gender'].value_counts())
print(df4['anxiety_status'].value_counts())

df5 = dataset[dataset['continent'] == 'AF']
print(df5['age'].mean(), df5['age'].std())
print(df5['gender'].value_counts())
print(df5['anxiety_status'].value_counts())

df6 = dataset[dataset['continent'] == 'OC']
print(df6['age'].mean(), df6['age'].std())
print(df6['gender'].value_counts())
print(df6['anxiety_status'].value_counts())

Filter the data to keep data that contains: Males and Females, over 18, have a region.

Then, remove data with countries that have strict data privacy laws.

In [ ]:
# Filter data
dataset = dataset.drop(dataset[(dataset['gender'] == 0) | (dataset['gender'] == 3)].index)  # Male and females only
dataset = dataset[dataset['age'] >= 18]  # Adults only
dataset = dataset[dataset['region'] != ""]  # Must have region

# Remove data with countries that have strict data privacy laws (for public use only)
dataset = dataset[dataset['continent'] != "EU"]
dataset = dataset[dataset['country'] != "CH"]
dataset = dataset[dataset['country'] != "IN"]
dataset = dataset[dataset['country'] != "JA"]
dataset = dataset[dataset['country'] != "AU"]

Run the cells below to see an overview of the data after filtering. 

In [ ]:
print("\nAfter filtering:")
print(dataset['gender'].value_counts())
print(dataset['agegroup'].value_counts())
print(dataset['age'].mean(), dataset['age'].std())
print(dataset['continent'].value_counts())
print(dataset['region'].value_counts())
print(dataset['anxiety_status'].value_counts())

In [ ]:
print("\nBreakdown by continent:")
df1 = dataset[dataset['continent'] == 'AS']
print(df1['age'].mean(), df1['age'].std())
print(df1['gender'].value_counts())
print(df1['anxiety_status'].value_counts())

df2 = dataset[dataset['continent'] == 'NA']
print(df2['age'].mean(), df2['age'].std())
print(df2['gender'].value_counts())
print(df2['anxiety_status'].value_counts())

df3 = dataset[dataset['continent'] == 'EU']
print(df3['age'].mean(), df3['age'].std())
print(df3['gender'].value_counts())
print(df3['anxiety_status'].value_counts())

df4 = dataset[dataset['continent'] == 'SA']
print(df4['age'].mean(), df4['age'].std())
print(df4['gender'].value_counts())
print(df4['anxiety_status'].value_counts())

df5 = dataset[dataset['continent'] == 'AF']
print(df5['age'].mean(), df5['age'].std())
print(df5['gender'].value_counts())
print(df5['anxiety_status'].value_counts())

df6 = dataset[dataset['continent'] == 'OC']
print(df6['age'].mean(), df6['age'].std())
print(df6['gender'].value_counts())
print(df6['anxiety_status'].value_counts())

Keep the columns that are necessary for DASS.

In [ ]:
# Drop unnecessary columns
to_drop = ["source", "screensize", "uniquenetworklocation", 
            "education", "urban", "engnat", "hand", "religion", 
            "orientation", "race", "voted", "married", "major",
            "introelapse", "testelapse", "surveyelapse", "familysize"]
dataset = dataset.drop(to_drop, axis=1)

for col in dataset.columns:
    if "TIPI" in col or "VCL" in col:
        dataset = dataset.drop([col], axis=1)
    elif col[0] == "Q" and (col[-1] == "E" or col[-1] == "I"):
        dataset = dataset.drop([col], axis=1)

Save the filtered dataset under data folder.

In [ ]:
# Saved filtered dataset
dataset.to_csv(os.path.join(data_folder, "data_filtered.csv"), index=None)

# 2. Data Preprocessing

Import the necessary libraries and define constants.

In [6]:
import numpy as np
import os
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

seed = 42
data_folder = "./data"

Define the preprocess function which will rebalance the minority class, one-hot encoding categorical variables, and z-score normalizing the numerical features.

Then, define the train_val_test_split function which will split and label the data into training, validation, and test datasets, which would be used for training the model, tuning model parameters, and evaluating the model's performance. 

In [7]:
def preprocess(data_df):
    """
    Pre-processing: rebalance, one-hot encode, normalize
    """
    # Separate majority and minority classes
    df_majority = data_df[data_df["anxiety_status"] == 1]
    df_minority = data_df[data_df["anxiety_status"] == 0]
    
    # Upsample minority class
    data_minority = resample(df_minority, 
                            replace=True,                       # sample with replacement
                            n_samples=len(df_majority.index),   # to match majority class
                            random_state=123)                   # reproducible results

    data_df = pd.concat([df_majority, data_minority])
    data_df = data_df.reset_index(drop=True)

    # Extract the label columns; separate features and labels
    labels_df = data_df[["anxiety_status"]].copy()
    feats_df = data_df.drop(["anxiety_score", "anxiety_status"], axis=1)

    # z-score normalization
    def z_score_norm(row, col, mean, stdev):
        z_score = (float(row[col]) - mean) / stdev
        return float(z_score)

    # One-hot encode gender and region
    label_encoder = LabelEncoder()
    oneh_encoder = OneHotEncoder()

    # Gender
    gender = label_encoder.fit_transform(feats_df["gender"])
    gender = pd.DataFrame(gender)
    gender = pd.DataFrame(oneh_encoder.fit_transform(gender).toarray())
    gender.columns = ["gender_m", "gender_f"]

    # Region
    region = label_encoder.fit_transform(feats_df["region"])
    region = pd.DataFrame(region)
    region = pd.DataFrame(oneh_encoder.fit_transform(region).toarray())
    region.columns = ["region_other", "region_east", "region_west"]

    # Combine and remove original columns
    feats_df = feats_df.drop(["gender", "country", "region", "agegroup", "continent"], axis=1)
    feats_df = pd.concat([feats_df, gender, region], axis=1)

    # One-hot encode question answers
    for col in feats_df.columns:
        if col[0] == "Q" and col[-1] == "A":
            temp = label_encoder.fit_transform(feats_df[col])
            temp = pd.DataFrame(temp)
            temp = pd.DataFrame(oneh_encoder.fit_transform(temp).toarray())

            col_names = []
            for c in temp.columns:
                col_names.append("{0}_{1}".format(col, c))
            temp.columns = col_names

            feats_df = feats_df.drop([col], axis=1)
            feats_df = pd.concat([feats_df, temp], axis=1)

    # Normalize numerical columns (Use z-score)
    mean = feats_df["age"].mean()
    stdev = feats_df["age"].std()
    feats_df["age_norm"] = feats_df.apply(
                    lambda row: z_score_norm(row, "age", mean, stdev), axis=1)
    feats_df = feats_df.drop(["age"], axis=1)

    return feats_df, labels_df


def train_val_test_split(feats_df, labels_df, rand=0, save=False):
    """
    Train / validation / test (holdout) dataset split
    """
    feats_arr = np.array(feats_df)
    labels_arr = np.array(labels_df)
    traintest_feats, valid_feats, traintest_labels, valid_labels = \
        train_test_split(feats_arr, labels_arr, test_size=0.10, random_state=seed)
    train_feats, holdout_feats, train_labels, holdout_labels = \
        train_test_split(traintest_feats, traintest_labels, test_size=0.1111, random_state=rand)

    train_feats = train_feats.astype(float)
    train_labels = train_labels.astype(float)
    valid_feats = valid_feats.astype(float)
    valid_labels = valid_labels.astype(float)
    holdout_feats = holdout_feats.astype(float)
    holdout_labels = holdout_labels.astype(float)

    if save:
        train_feats.to_csv(os.path.join(data_folder, "train_feats.csv"), index=None)
        train_labels.to_csv(os.path.join(data_folder, "train_labels.csv"), index=None)
        valid_feats.to_csv(os.path.join(data_folder, "valid_feats.csv"), index=None)
        valid_labels.to_csv(os.path.join(data_folder, "valid_labels.csv"), index=None)
        holdout_feats.to_csv(os.path.join(data_folder, "holdout_feats.csv"), index=None)
        holdout_labels.to_csv(os.path.join(data_folder, "holdout_labels.csv"), index=None)

    return train_feats, train_labels, valid_feats, valid_labels, holdout_feats, holdout_labels

Preprocess the filtered data by calling the preprocess function, and save the preprocessed data in the data_folder.

In [8]:
data = pd.read_csv(os.path.join(data_folder, "data_filtered_1000.csv")) # Using the sample dataset. Change the file name accordingly if using another dataset. 
feats_df, labels_df = preprocess(data)

feats_df.to_csv(os.path.join(data_folder, "features.csv"), index=None)
labels_df.to_csv(os.path.join(data_folder, "labels.csv"), index=None)